In [2]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import pdal
from dask.distributed import Client

np.set_printoptions(precision=4, suppress=True)

from forest_structure_tools.metrics import forest_structure_metrics

In [3]:
data_dir = Path("../data")
plots_dir = data_dir / "outputs" / "plots"
plots_lidar_dir = plots_dir / "lidar"

plots_gdf = gpd.read_file(plots_dir / "plots.geojson")
plot_ids = plots_gdf['id'].to_list()

def read_plot_lidar(plot_id: str):
    pl = pdal.Reader(str(plots_lidar_dir / f"{plot_id}.copc.laz")).pipeline()
    pl.execute()

    return pl.arrays[0]

In [4]:
metrics_dir = plots_dir / "metrics"
metrics_dir.mkdir(parents=True, exist_ok=True)

In [60]:
def add_plot_dimension_to_ds(ds: xr.Dataset, plot_id: str) -> xr.Dataset:
    site = plot_id[:-3]
    site_type = site[:3]

    ds = ds.expand_dims(plot=[plot_id]).assign_coords(
        plot=("plot", [plot_id]), site=("plot", [site]), site_type=("plot", [site_type])
    )

    return ds


def calculate_metrics(plot_id: id, xy_bin_size: float | None = None, z_bin_size: float | None = 1, weighted = False):
    points = read_plot_lidar(plot_id)
    x = points["X"]
    y = points["Y"]
    z = points["Z"]

    if weighted:
        weights = 1 / points["NumberOfReturns"]
    else:
        weights = None

    metrics = forest_structure_metrics(x=x, y=y, z=z, xy_bin_size=xy_bin_size, z_bin_size=z_bin_size, weights=weights)


    if xy_bin_size is not None:
        x_offset = metrics.x.min().item()
        y_offset = metrics.y.min().item()

        metrics['x_offset'] = x_offset
        metrics['y_offset'] = y_offset

        metrics['x'] = metrics['x'] - x_offset
        metrics['y'] = metrics['y'] - y_offset


    metrics = add_plot_dimension_to_ds(metrics, plot_id)

    metrics.attrs["xy_bin_size"] = str(xy_bin_size)
    metrics.attrs["z_bin_size"] = str(z_bin_size)
    metrics.attrs["weighted"] = str(weighted)
    
    return metrics


In [6]:
calculate_metrics("AGG_O_01_P1", xy_bin_size=5, z_bin_size=1, weighted=False)

<xarray.Dataset> Size: 138kB
Dimensions:              (x: 14, y: 14, plot: 1, z: 18)
Coordinates:
  * x                    (x) float64 112B 4.63e+05 4.63e+05 ... 4.63e+05
  * y                    (y) float64 112B 5.26e+06 5.26e+06 ... 5.26e+06
  * z                    (z) float64 144B 0.0 1.0 2.0 3.0 ... 15.0 16.0 17.0
  * plot                 (plot) <U11 44B 'AGG_O_01_P1'
    site                 (plot) <U8 32B 'AGG_O_01'
    site_type            (plot) <U3 12B 'AGG'
Data variables: (12/37)
    max                  (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    min                  (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    range                (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    mean                 (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    median               (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    sd                   (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    ...                   ...
    pai                  (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    fhd                  (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    norm_fhd             (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    cv_inside_p          (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    cv_ppi               (plot, x, y) float64 2kB nan nan nan ... nan nan nan
    cv_pad               (plot, x, y) float64 2kB nan nan nan ... nan nan nan
Attributes:
    xy_bin_size:  5
    z_bin_size:   1
    weighted:     False

In [7]:
client = Client()  # Start a Dask client
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52008,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:52021,Total threads: 2
Dashboard: http://127.0.0.1:52024/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52011,


In [8]:
futures = client.map(calculate_metrics, plot_ids, key=plot_ids, xy_bin_size=None, z_bin_size=1, weighted=False)
results = client.gather(futures)

In [9]:
ds = xr.concat(results, dim='plot')
ds

<xarray.Dataset> Size: 933kB
Dimensions:              (plot: 315, z: 108)
Coordinates:
  * z                    (z) float64 864B 0.0 1.0 2.0 3.0 ... 105.0 106.0 107.0
  * plot                 (plot) <U12 15kB 'AGG_O_01_P1' ... 'ULY_Y_96_P5'
    site                 (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96'
    site_type            (plot) <U3 4kB 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'
Data variables: (12/37)
    max                  (plot) float64 3kB 16.28 17.18 21.46 ... 19.32 35.28
    min                  (plot) float64 3kB 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    range                (plot) float64 3kB 16.28 17.18 21.46 ... 19.32 35.28
    mean                 (plot) float64 3kB 4.428 4.471 5.346 ... 1.36 1.908
    median               (plot) float64 3kB 4.555 4.654 5.666 ... 0.56 0.376
    sd                   (plot) float64 3kB 4.093 4.085 4.651 ... 2.601 4.168
    ...                   ...
    pai                  (plot) float64 3kB 1.533 1.441 1.663 ... 1.711 1.318
    fhd                  (plot) float64 3kB 2.384 2.406 2.556 ... 1.551 1.732
    norm_fhd             (plot) float64 3kB 0.8247 0.8171 ... 0.5093 0.4795
    cv_inside_p          (plot) float64 3kB 1.051 1.099 1.106 ... 2.498 3.2
    cv_ppi               (plot) float64 3kB 0.2682 0.2544 ... 0.2848 0.1992
    cv_pad               (plot) float64 3kB 1.574 1.397 1.646 ... 3.604 4.62
Attributes:
    xy_bin_size:  None
    z_bin_size:   1
    weighted:     False

In [10]:
ds.to_netcdf(metrics_dir / "metrics_grid_none_z_1m.nc")

In [11]:
client.close()

# Gridded Metrics

## 10m Grid

In [68]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52561,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:52573,Total threads: 2
Dashboard: http://127.0.0.1:52577/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52564,


In [69]:
futures = client.map(calculate_metrics, plot_ids, key=plot_ids, xy_bin_size=10, z_bin_size=1, weighted=False)
results = client.gather(futures)

In [70]:
ds = xr.concat(results, dim='plot')
ds.to_netcdf(metrics_dir / "metrics_grid_10m_z_1m.nc")
ds

<xarray.Dataset> Size: 73MB
Dimensions:              (plot: 315, x: 9, y: 9, z: 108)
Coordinates:
  * x                    (x) float64 72B 0.0 10.0 20.0 30.0 ... 60.0 70.0 80.0
  * y                    (y) float64 72B 0.0 10.0 20.0 30.0 ... 60.0 70.0 80.0
  * z                    (z) float64 864B 0.0 1.0 2.0 3.0 ... 105.0 106.0 107.0
  * plot                 (plot) <U12 15kB 'AGG_O_01_P1' ... 'ULY_Y_96_P5'
    site                 (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96'
    site_type            (plot) <U3 4kB 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'
Data variables: (12/39)
    max                  (plot, x, y) float64 204kB nan nan 12.95 ... nan nan
    min                  (plot, x, y) float64 204kB nan nan 0.0 ... nan nan nan
    range                (plot, x, y) float64 204kB nan nan 12.95 ... nan nan
    mean                 (plot, x, y) float64 204kB nan nan 5.278 ... nan nan
    median               (plot, x, y) float64 204kB nan nan 5.205 ... nan nan
    sd                   (plot, x, y) float64 204kB nan nan 3.39 ... nan nan nan
    ...                   ...
    norm_fhd             (plot, x, y) float64 204kB nan nan 0.9056 ... nan nan
    cv_inside_p          (plot, x, y) float64 204kB nan nan 0.6674 ... nan nan
    cv_ppi               (plot, x, y) float64 204kB nan nan 0.4595 ... nan nan
    cv_pad               (plot, x, y) float64 204kB nan nan 2.334 ... nan nan
    x_offset             (plot) float64 3kB 4.63e+05 4.631e+05 ... 4.552e+05
    y_offset             (plot) float64 3kB 5.26e+06 5.26e+06 ... 5.276e+06
Attributes:
    xy_bin_size:  10
    z_bin_size:   1
    weighted:     False

In [73]:
client.close()

## 5m Grid

In [74]:
client = Client()
client

/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52653 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52653/status,
Dashboard: http://127.0.0.1:52653/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52654,Workers: 0
Dashboard: http://127.0.0.1:52653/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:52665,Total threads: 2
Dashboard: http://127.0.0.1:52668/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52657,


In [75]:
futures = client.map(calculate_metrics, plot_ids, key=plot_ids, xy_bin_size=5, z_bin_size=1, weighted=False)
results = client.gather(futures)

In [76]:
ds = xr.concat(results, dim='plot')
ds.to_netcdf(metrics_dir / "metrics_grid_5m_z_1m.nc")
ds

<xarray.Dataset> Size: 261MB
Dimensions:              (plot: 315, x: 17, y: 17, z: 108)
Coordinates:
  * x                    (x) float64 136B 0.0 5.0 10.0 15.0 ... 70.0 75.0 80.0
  * y                    (y) float64 136B 0.0 5.0 10.0 15.0 ... 70.0 75.0 80.0
  * z                    (z) float64 864B 0.0 1.0 2.0 3.0 ... 105.0 106.0 107.0
  * plot                 (plot) <U12 15kB 'AGG_O_01_P1' ... 'ULY_Y_96_P5'
    site                 (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96'
    site_type            (plot) <U3 4kB 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'
Data variables: (12/39)
    max                  (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    min                  (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    range                (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    mean                 (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    median               (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    sd                   (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    ...                   ...
    norm_fhd             (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    cv_inside_p          (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    cv_ppi               (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    cv_pad               (plot, x, y) float64 728kB nan nan nan ... nan nan nan
    x_offset             (plot) float64 3kB 4.63e+05 4.631e+05 ... 4.552e+05
    y_offset             (plot) float64 3kB 5.26e+06 5.26e+06 ... 5.276e+06
Attributes:
    xy_bin_size:  5
    z_bin_size:   1
    weighted:     False

In [77]:
client.close()

## 1m Grid

This may be pushing it a bit

In [78]:
client = Client()
client

/Users/jgregory/Code/forest-structure-tools/.pixi/envs/default/lib/python3.13/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52742 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:52742/status,
Dashboard: http://127.0.0.1:52742/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52743,Workers: 0
Dashboard: http://127.0.0.1:52742/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:52757,Total threads: 2
Dashboard: http://127.0.0.1:52760/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52746,


In [79]:
futures = client.map(calculate_metrics, plot_ids, key=plot_ids, xy_bin_size=1, z_bin_size=1, weighted=False)
results = client.gather(futures)

In [80]:
ds = xr.concat(results, dim='plot')
ds.to_netcdf(metrics_dir / "metrics_grid_1m_z_1m.nc")
ds

<xarray.Dataset> Size: 6GB
Dimensions:              (plot: 315, x: 79, y: 82, z: 108)
Coordinates:
  * x                    (x) float64 632B 0.0 1.0 2.0 3.0 ... 76.0 77.0 78.0
  * y                    (y) float64 656B 0.0 1.0 2.0 3.0 ... 79.0 80.0 81.0
  * z                    (z) float64 864B 0.0 1.0 2.0 3.0 ... 105.0 106.0 107.0
  * plot                 (plot) <U12 15kB 'AGG_O_01_P1' ... 'ULY_Y_96_P5'
    site                 (plot) <U9 11kB 'AGG_O_01' 'AGG_O_01' ... 'ULY_Y_96'
    site_type            (plot) <U3 4kB 'AGG' 'AGG' 'AGG' ... 'ULY' 'ULY' 'ULY'
Data variables: (12/39)
    max                  (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    min                  (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    range                (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    mean                 (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    median               (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    sd                   (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    ...                   ...
    norm_fhd             (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    cv_inside_p          (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    cv_ppi               (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    cv_pad               (plot, x, y) float64 16MB nan nan nan ... nan nan nan
    x_offset             (plot) float64 3kB 4.63e+05 4.631e+05 ... 4.552e+05
    y_offset             (plot) float64 3kB 5.26e+06 5.26e+06 ... 5.276e+06
Attributes:
    xy_bin_size:  1
    z_bin_size:   1
    weighted:     False

In [84]:
client.close()